In [1]:
#PDF RAG QnA System

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from langchain_community.document_loaders import PyPDFLoader

file_path = 'Analytical Science Advances - 2023 - Rodriguez - Recent advances of Raman spectroscopy for the analysis of bacteria.pdf'
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

15


In [4]:
print(docs[0].page_content[0:200])
print(docs[0].metadata)

81 Analytical Science AdvancesMini Review
doi.org/10.1002/ansa.202200066
Received: 26 December 2022
Revised: 3 March 2023
Accepted: 10 March 2023
Recent advances of Raman spectroscopy for the analysis
{'source': 'Analytical Science Advances - 2023 - Rodriguez - Recent advances of Raman spectroscopy for the analysis of bacteria.pdf', 'page': 0}


In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

In [6]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=5000, chunk_overlap= 1000)
splits = text_splitter.split_documents(docs)
embed = OpenAIEmbeddings(show_progress_bar= True, request_timeout= 10000 )
vectorstore = Chroma.from_documents(documents= splits, embedding= embed, persist_directory= './c_db')

retriever = vectorstore.as_retriever()

  0%|          | 0/1 [00:00<?, ?it/s]

In [13]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [14]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "The End!", in a new line, at the end of the answer.

{context}

Question: {question}

Helpful Answer:"""

custom_rag_prompt = PromptTemplate.from_template(template)

rag_chain = ({"context": retriever | format_docs, "question": RunnablePassthrough()} | custom_rag_prompt | llm | StrOutputParser())


In [15]:
response = rag_chain.invoke("Eplain how Raman spectroscopy is used for the analysis of microorganisms")

  0%|          | 0/1 [00:00<?, ?it/s]

In [16]:
response

'Raman spectroscopy is used for the analysis of microorganisms by measuring the inelastic light scattering from a laser source interacting with the molecules in the sample. This technique produces specific fingerprinting spectra that enable the identification and discrimination of different bacterial genera, species, and strains based on their vibrational modes. Advanced data processing methods like chemometrics and machine learning algorithms are often employed to interpret the Raman spectral data and enhance the differentiation capabilities. \n\nThe End!'